In [ ]:
%load_ext autoreload
%autoreload 2

# Sonecule: DiscretePMS – Discrete Parameter-Mapping Sonification

This notebook introduces and demonstrates usage of the `DiscretePMS` sonecule.
* The sonecule spawns an instance of a synth for each data point in a data set
* The synth offers several parameters that can set (only) at init time
* Specifically the parameters are:
  * amplitude
  * frequency
  * sharpness
  * spatial panning
  * attack time
  * duration of the event
  * release time
* In many custom situations, users will wish to create a custom synth, so this can be passed on for individual sonifications, then replacing the default synth.
* The mapping specifies how available data channels shall control the individual parameters of the synthesizer. A readable and quite flexible mapping specification standard is offered to control mapping function, ranges, and even pre- and post-mapping warpings, as demonstrated and explained below.

In [ ]:
# headers and imports for the demo
import sonecules as sn
from pya import Asig
import pyamapping as pam
import matplotlib.pyplot as plt

# setup for matplotlib 
plt.rcParams["figure.figsize"] = (8,3)
%matplotlib widget

# start sonecules (with default backend sc3nb, aka sc3)
sn.startup()
ctx = sn.gcc()  # get the context as ctx

Load data sets used for the demo

In [ ]:
%run ../data/prepare-data.ipynb

We use the penguins dataframe for the examples. Here is a plot of the features.

In [ ]:
sns.pairplot(data=penguins_df, hue="species", height=1.2);

Let's also store it in df and list the available features

In [ ]:
df = dataframes['penguins']
df.columns

## Usage Demo for the DiscretePMS Sonecule

In [ ]:
from mesonic.synth import Synth
from sonecules.scoreson import DiscretePMS, pms

The following code cell shows everything needed 
- to create the sonecule with data, 
- to clear the auditory canvas (aka timeline)
- to start the playback at a given rate
- to plot the timeline.

Usually we want to use a specific synths which we would define using the backend.
- We can create it using `context.synths.add_synth_def`, which in this case creates the defines the synth for the default backend (sc3nb)
- The synth definition process is currently in active development and will improve in the future.  
- However our ultimate goal is that sonecules will already offer a well curated library of pre-packaged synths, so that it is rarely necessary to craft your own.

The default synth (if none is provided is "cpmssyn", a 'continuous synth for PMSon'). 
- It is a pitched tone with added vibrato 
- It offers the continuous controls:

| parameter | range      | meaning                                         |
| --------- | ---------- | ----------------------------------------------- |
| freq      | [20,0.5sr] | frequency [Hz]                                  |
| amp       | [0, 1]     | amplitude                                       |
| sharp     | [0, 10]    | sharpness, the higher the more sharp the sound  |
| pan       | [-1, 1]    | spatial panning from left to right              |
| dur       | [0,...]    | duration in seconds                             |
| att       | [0,...]    | attack time (<dur)                              |
| rel       | [0,...]    | release time (<dur)                             |
| vibfreq   | [0,...]    | vibrato frequency [Hz]                          |
| vibir     | [0,...]    | relative vibrato intensity (dfreq = vibir*freq) |

In most cases we want to use a custom synths, so let's see how this is achieved right at the beginning.
- for future sonecules releases we plan to offer a library of curated default synths for various backends so that end users won't need to be bothered with such details.
- for this demo we use the sc3nb backend, hence our synth definition takes the shape of a Supercollider SynthDef, consisting of name and definition function in sc3.

In [ ]:
ctx.synths.add_synth_def("syndpms", 
"""{ | out=0, freq=400, dur=0.4, att=0.001, rel=0.5, amp=0.1, vibfreq=0, vibir=0, sharp=0, pan=0 | 
    var vib = SinOsc.ar(vibfreq, mul: vibir*freq, add: freq);
    var sig = HPF.ar(Formant.ar(vib, vib, bwfreq: vib*(sharp+1), mul: amp), 40);
    var env = EnvGen.kr(Env.new([0,1,1,0], [att, dur-att-rel, rel]), doneAction:2);
    Out.ar(out, Pan2.ar(sig, pan, env));
}""")

Now let's create a synths as immutable

In [ ]:
synth = ctx.synths.create("syndpms", mutable=False)

and orchestrate a few tones...
- btw: this is basic mesonic functionality, so check the mesonic tutorials, in case you'd like to familiarize deeper with using synths in mesonic.

In [ ]:
with ctx.test():  # demo for the syndpms, change the params and execute again
    for i in range(3):
        with ctx.at(i/10) as timepoint:
            synth.start(freq=280 + 100 * i, dur=i/3, amp=0.1, 
                att=0, rel=0.1, sharp=5, vibfreq=5, vibir=0.02, pan=0)

In [ ]:
ctx.reset()  # clear the timeline after the test

The next cell shows all steps in specifying the mapping in one go

In [ ]:
df.columns

The mapping is just a dictionary where 
- keys are the parameters, and 
- values is either **a number** (for a constant) or **a dictionary** with some mandatory and some optional keys:
    - "col": column/feature of the dataset to be used 
    - "fn": mapping function(values, xr, yr), but strings such as "lin", "exp", "log"
        are allowed as shortcut
    - "yr": the target range for the parameter (implied from synths bounds if omitted)
    - "xr": the source range (which is implied from the data if omitted)
see documenation for further mapping flags


In [ ]:
# create a sonecule, initialized with the synth to be used
# second argument can provide defaults and bounds for parameters
sndpms = DiscretePMS("syndpms")

# the most flexible and raw definition as dictionary
test_mapping = {
    "onset" : {"col": "bill_length_mm",    "fn": "lin", "yr": [0,8]},
    "freq"  : {"col": "flipper_length_mm", "fn": "exp", "yr": [pam.midi_to_cps(40), pam.midi_to_cps(110)]},
    "sharp" : {"col": "body_mass_g",       "fn": "lin", "yr": [1, 4]},
    "pan" : 0, 
    "att" : 0, 
    "dur" : 0.25, 
    "rel" : 0.15, 
    "amp" : 0.1,
}

# render the sonification into the timeline
sndpms.reschedule(df=df, mapping=test_mapping)

# start the realtime playback at a given rate
sndpms.start(rate=1)

Usually this can be done in a more condensed way, by
- omitting the defaults and bounds (later synths will come with good such values anyway)
- using the `(column, mapping_function, yr)` tuple instead of a value dictionary
  - e.g. `['flipper_length_mm', 'exp', [440, 880]]`
- using the dict() function for the outer dict, which allows to write a dict as function kwargs
- using the **parameter mapping specification** function `pms()` for the inner (d.h. value) dicts,
- starting the sonification by daisy-chaining of the start() method
- by removing the constructor for subsequent calls, i.e. reusing the sonecule sndpms so that only `sndpms.schedule().start()` is called

We apply most of these tipps, but for the mapping only condense by using the value tuple 

In [ ]:
sndpms.reschedule(df=df, mapping={
    "onset" : ("bill_length_mm",    "lin", [0, 2]),
    "freq"  : ("flipper_length_mm", "exp", [pam.midi_to_cps(40), pam.midi_to_cps(110)]),
    "sharp" : ("body_mass_g",       "lin", [1, 4]),
    "pan" : 0, "att" : 0, "dur" : 0.25, "rel" : 0.15, "amp" : 0.1,
}).start(rate=1)

No let's see how to use dict with keyword arguments, allowing to get rid of they quotes for the keys and enhance readability

In [ ]:
sndpms.reschedule(df=df, mapping=dict(
    onset = ("bill_length_mm",    "lin", [0, 2]),
    freq  = ("flipper_length_mm", "exp", [pam.midi_to_cps(40), pam.midi_to_cps(110)]),
    sharp = ("body_mass_g",       "lin", [1, 4]),
    pan = 0, att = 0, dur = 0.25, rel = 0.15, amp = 0.1
)).start(rate=1)

Finally let's use the helper `pms` (parameter mapping specification), which eases the creation of parameter specification dictionary and flexibly allows to add features.
Note that in the following example 
- we specify the source range (xr) for the flipper_length range to xr=[190, 195]
- this is mapped to frequency and due to the clipping, we hear how many penguins
  - have lower than 190 mm flipper lengths (200 Hz tone)
  - have higher than 195 mm flipper length (400 Hz tone) 

In [ ]:
sndpms.reschedule(df=df, mapping=dict(
    onset = pms("bill_length_mm",    "lin", [0, 3]),
    freq  = pms("flipper_length_mm", "exp", [200, 400], xr=[190, 195], clip="minmax"),
    sharp = pms("body_mass_g",       "lin", [1, 4]),
    pan = 0, att = 0, dur = 0.05, rel = 0.15, amp = 0.1
)).start(rate=1)

Best is learning by doing: play with the following copied code cell to experiment with the mappings and mapping specifications

In [ ]:
sndpms.reschedule(df=df, mapping=dict(
    onset = pms("bill_length_mm",    "lin", [0, 3]),
    freq  = pms("flipper_length_mm", "exp", [200, 400], xr=[190, 195], clip="minmax"),
    sharp = pms("body_mass_g",       "lin", [1, 4]),
    pan = 0, att = 0, dur = 0.05, rel = 0.15, amp = 0.1
)).start(rate=1)

For the dictionary synonymous keys can be used, defaulting specs to the first
* `'col'`: to specify the name of the pandas series uses as data for mapping
  * equivalent keys are `'n', 'name', 'feature', 'feat', 'f'`
* `'fn'`: to specify the mapping function to map from data feature to parameter 
  * equivalent key is `'via'`
  * supported values are: `"lin", "exp", "log"` 
  * not yet supported: `"poly({{n}})"`
* `'yr'`: to specify the target range `[y1, y2]` - can be unsorted if needed
  * equivalent keys are 'to' and 'yrange'
* `'xr'`:  to specify the source (data) range as `[min, max]` tuple
  * equivalent keys are `'within'` and `'xrange'`
* `'xqr'`: to specify the source range in quantiles
  * equivalent keys are `within_q` and `xqrange`
  * not yet supported
* `'clip'` to specify how mapping results are clipped
  * values are `"min", "max", "minmax", "" or None` (the latter: TBC)
  * ToDo: only minmax supported so far
* `'pre'` to specify one or many functions on the series to be performed prior to mapping
  * values are either strings such as `midi_to_cps, cps_to_midi, amp_to_db, db_to_amp, floor, diff` (and soon a few more)
  * value can also be a list of such strings: execution is in order
  * instead of strings, functions (names or lambda expressions) can be specified
* `'post'` to specify one or many functions to modify the resulting series after mapping
  * see notes for `'pre'` for syntax
  * as example, 
    * instead of `fn="exp", yr=[pam.midi_to_cps(40), pam.midi_to_cps(60)]`
    * you could use `fn="lin", yr=[40, 60], post="midi_to_cps"`

In [ ]:
# example how the pms() helper function turns a specification into the standard (less readable dict form)
pms("bill_length_mm",    "lin", yr=[0, 12])

In [ ]:
# variant 1: map_function:
# pms would return a dictionary, depending on call arguments, shortcuts defined for arguments such as fn
sndpms.reschedule(df=df, mapping=dict(
    onset = pms("bill_length_mm",    "lin", yr=[0, 12]),
    freq  = pms("flipper_length_mm", "exp", yr=[pam.midi_to_cps(30), pam.midi_to_cps(94)]),
    sharp = pms("body_mass_g",       "lin", yr=[1, 15]),
    pan   = pms("bill_depth_mm",     "lin", xr=[15, 16], yr=[-1, 1]), 
    att = 0.3,  dur = 1.0, rel = 0.8, amp = 0.3,
)).start()

In [ ]:
# variant 2: map_function with dict (to be more pythonic)
# pms would return a dictionary, depending on call arguments
sndpms.reschedule(df=df, mapping=dict(
    onset = pms("INDEX", "lin", yr=[0, 8]),
    freq  = pms("flipper_length_mm", "exp", yr=[pam.midi_to_cps(40), pam.midi_to_cps(110)]),
    sharp = pms("body_mass_g", "lin", yr=[1, 4]),
    pan   = pms("bill_depth_mm", xr=[13, 30], fn="lin", yr=[-1, 1]), 
    att=0, dur=0.02, amp=0.1, rel=0
)).start()

In [ ]:
# variant 3: map_function with dict (to look more pythonic)
# pms args would be taken as col, xr=None, fn="lin", yr=None (in this order) 
sndpms.reschedule(df=df, mapping=dict(
    onset = pms("body_mass_g", fn="lin", yr=[2, 8]),
    freq  = pms(col="flipper_length_mm", fn="exp", yr=[pam.midi_to_cps(20), pam.midi_to_cps(70)]),
    sharp = pms(col="body_mass_g", fn="lin", yr=[1, 4]),
    pan   = pms(col="bill_depth_mm", xqr=[0.2, 0.8], fn="lin", yr=[-1, 1]), 
    att=0, dur=0.05, amp=0.1, rel=0.25,
)).start()

In [ ]:
# for own mapping experiments, its useful to see all columns
df.columns

In [ ]:
sndpms.reschedule(df=df, mapping=dict(
    # mapping target = ('data column', mapping function, target range [from, to])
    onset = ("bill_length_mm", "lin", [0, 5]),
    freq  = ("body_mass_g", "exp", [pam.midi_to_cps(40), pam.midi_to_cps(110)]),
    sharp = ("bill_depth_mm", "lin", [1, 2]),
    rel   = ("bill_length_mm", "lin", [0.3, 1.5]), 
    pan=0, att=0, dur=0.01, amp=0.1,
)).start()

### One-To-Many Mapping: 

So far we have demonstrated one-to-one mappings where different data channels control different parameters in a 1-to-1 manner.
Let's look into other mapping types.

Example using a 1-channel ECG
- using the one-to-many mapping to give more saliency to  variations at different values (e.g., R-Peak, T-Wave, Iso-Electricity, Negative values)


In [ ]:
ecg_df = dataframes['ecg'][[0,5]]
ecg_df.plot(figsize=(10,2));

Let's use one channel (channel 0), both for a mapping to frequency, amplitude, sharpness and relative length, all using its own mapping function. In doing so we can let different features convey variations in different ranges

In [ ]:
sndpms.reschedule(df=ecg_df, mapping=dict(
    onset = pms("INDEX", "lin", [0, 10]),
    freq  = pms(0, "lin", [60, 84], pre=["diff", "abs"], xr=[0, 0.3], post=[lambda x: np.round(x/5)*5, "midicps"]),
    amp   = pms(0, "lin", [-20, -5], pre="abs", xr=[0, 1], post="dbamp", clip="minmax"),
    sharp = pms(0, "lin", [0, 2],    xr=[0,  0.2], clip="minmax"),
    rel   = pms(0, "exp", [0.01, 5], xr=[0.25, 1], clip="minmax"),
    pan   = 0,
    att   = 0.01,
    dur   = 0,
)).start()

Note that we can inspect the mapping parameters via the sonecules mapping_df attribute

In [ ]:
sndpms.mapping_df.plot(subplots=True, figsize=(8,4));

In [ ]:
ecg_df.columns

To demonstrate the flexibility, here is another example, where the mapping uses both channels 0 and 5 for several parameters

In [ ]:
sndpms.reschedule(df=ecg_df, mapping=dict(
    onset = pms(0, "lin", [0.05,   10]),
    freq  = pms(5, "exp", [ 200, 1200]),
    amp   = pms(5, "exp", [0.05,  0.5], xr=[0.25, 1],   clip="minmax"),
    sharp = pms(5, "lin", [   0,    2], xr=[   0, 0.2], clip="minmax"),
    rel   = pms(0, "exp", [0.02,    5], xr=[   0, 1],   clip="minmax"),
    pan   = 0,
    att   = 0,
    dur   = 0,
)).start()

ecg_df.plot(x=0, y=5, figsize=(3,3), lw=0.1, marker=".", ms=0.8);
# sndpms.mapping_df.plot(subplots=True, figsize=(8,4));

In [ ]:
sndpms.remove()  # does remove the events from the sonecule from the Timeline
ctx.timeline

## Code Template

The following code snippets are intended for copy & paste to your notebooks, to facilitate getting your data sonified
using this sonecule.

In [ ]:
# load your multi-channel data into an Asig, e.g. 
data = np.random.random((40, 4))-0.5 # 100 rows with 8 channels, here same fake data
data = np.cumsum(data,axis=0)
df = pd.DataFrame(data, columns=["c1", "c2", "c3", "c4"])
df.plot(subplots=True);
df.head()

In [ ]:
ctx.synths.add_synth_def("syd0", 
"""{ | out=0, freq=400, dur=0.4, att=0.001, rel=0.5, amp=0.1, vibfreq=0, vibir=0, sharp=0, pan=0 | 
    var vib = SinOsc.ar(vibfreq, mul: vibir*freq, add: freq);
    var sig = HPF.ar(Formant.ar(vib, vib, bwfreq: vib*(sharp+1), mul: amp), 40);
    var env = EnvGen.kr(Env.new([0,1,1,0], [att, dur-att-rel, rel]), doneAction:2);
    OffsetOut.ar(out, Pan2.ar(sig, pan, env));
}""")

# load your data / select your data
mydf = df

# sonecule for your synth with defaults and bounds
sndpms = DiscretePMS("syd0")

In [ ]:
# example custom function to quantize pitch mapping to grid of semitones
def myfreqfn(v, xr, yr, q=4):
    return pam.midi_to_cps(np.round(pam.linlin(v, *xr, *yr)/q)*q)

# and render the sonification into the timeline
sndpms.reschedule(df=mydf, mapping=dict(
    onset = pms("INDEX", "lin", [0, 3]),
    freq  = pms("c3", myfreqfn, [40, 70], q=7), # try q=7, q=12, q=1
    sharp = pms("c2", "lin", [0, 5]),
    dur   = pms("c1", "exp", [0.05, 1]),
    rel   = pms("c1", "exp", [0.05, 1]), 
    amp   = pms("c4", "exp", [0.9, 0.01]),
    pan = 0, att = 0,
))

# finally start the realtime playback at a given rate'
sndpms.start(rate=1)

# if needed: plot the timeline using 
ctx.timeline.plot()

In [ ]:
ctx.close()  # close the mesonic context, exits backend gracefully